# 02 Training Periods Experiments

In [3]:
from pathlib import Path
import os
import warnings

%load_ext autoreload
%autoreload

# ignore warnings for now ...
warnings.filterwarnings('ignore')

if Path('.').absolute().parents[1].name == 'ml_drought':
    os.chdir(Path('.').absolute().parents[1])

!pwd

/Users/tommylees/github/ml_drought


In [6]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

data_dir = Path('data/')
# data_dir = Path('/Volumes/Lees_Extend/data/zip_data')
data_dir = Path('/Volumes/Lees_Extend/data/ecmwf_sowc/data/')

assert data_dir.exists()

In [3]:
import seaborn as sns
from src.utils import drop_nans_and_flatten

from src.analysis import read_train_data, read_test_data, read_pred_data

# Read in the data

Get all dataloaders from the train/test data
```python
next(iter(dataloader))
```

In [4]:
X_train, y_train = read_train_data(data_dir)
X_test, y_test = read_test_data(data_dir)

In [5]:
ds = xr.merge([y_train, y_test]).sortby('time').sortby('lat')
d_ = xr.merge([X_train, X_test]).sortby('time').sortby('lat')
ds = xr.merge([ds, d_])

In [6]:
from src.utils import get_ds_mask
mask = get_ds_mask(X_train.VCI)

## Read in a model for some parameters/Options

In [7]:
from src.models import load_model
ealstm_eg = load_model(
    data_dir / 'models' / '02_training_periods' / '2020_01_11:175753_one_month_forecast_TRhigh_TEhigh_LEN5' / 'ealstm' / 'model.pt'
)

In [15]:
print(dir(ealstm_eg), '\n\n')

print(ealstm_eg.ignore_vars,)
print(ealstm_eg.hidden_size,)
print(ealstm_eg.data_vars,)
print(ealstm_eg.static_embedding_size,)
print(ealstm_eg.static_size,)
print(ealstm_eg.surrounding_pixels,)
print(ealstm_eg.rnn_dropout,)

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_concatenate_data', '_convert_delta_to_raw_values', '_get_background', '_get_morris_explanations', '_get_shap_explanations', '_initialize_model', '_input_to_tuple', '_make_analysis_folder', '_one_hot', 'batch_size', 'current_size', 'data_path', 'data_vars', 'dense_features', 'device', 'evaluate', 'experiment', 'explain', 'explainer', 'features_per_month', 'get_dataloader', 'hidden_size', 'ignore_vars', 'include_latlons', 'include_monthly_aggs', 'include_pred_month', 'include_yearly_aggs', 'input_dense', 'load', 'model', 'model_dir', 'model_name', 'models_dir', 'num_locations', 'pred_months', 'predict', 'predict_delta', 'rnn_dro

# Read in results

In [4]:
from scripts.utils import get_results

